In [1]:
import pandas as pd
import numpy as np

import re


import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### Города и поселки ###

In [2]:
# загружаем все города и поселки

all_cities = pd.read_excel('C:/00_Projects/Lenin/Datasets/01_FIAS/03_all_cities.xlsx', sheet_name='cities')

### Улицы ###

In [3]:
# загружаем все улицы

work_streets = pd.read_csv('C:/00_Projects/Lenin/Datasets/01_FIAS/05_all_streets.csv')

In [4]:
# загружаем ленинские улицы (включая косвенные)

lenin_streets_full = pd.read_excel('C:/00_Projects/Lenin/Datasets/01_FIAS/06_lenin_streets.xlsx', sheet_name='full')

In [5]:
# загружаем ленинские улицы (только точные)

lenin_streets_strict = pd.read_excel('C:/00_Projects/Lenin/Datasets/01_FIAS/06_lenin_streets.xlsx', sheet_name='lenin')

In [6]:
# загружаем ленинские улицы (без переулков)

lenin_only_streets = pd.read_excel('C:/00_Projects/Lenin/Datasets/01_FIAS/06_lenin_streets.xlsx', sheet_name='lenin_os')

### Памятники ###

In [7]:
# Загружаем памятники из ОКН

lenin_monuments_OKN = pd.read_excel('C:/00_Projects/Lenin/Datasets/02_Monuments/08_lenin_reestrOKN.xlsx',
                                     sheet_name='need')

In [8]:
# Загружаем памятники из базы leninstatues

lenin_monuments_lst = pd.read_excel('C:/00_Projects/Lenin/Datasets/02_Monuments/09_lenin_monuments.xlsx',
                                     sheet_name='need')

### Столицы регионов ###

In [9]:
# загружаем столицы регионов

reg_capitals = pd.read_excel('C:/00_Projects/Lenin/Datasets/03_Other/region_name.xlsx', sheet_name='reg_capitals')

### Города с населением больше 100 тыс. ###

In [10]:
cities_100 = pd.read_excel('C:/00_Projects/Lenin/Datasets/03_Other/cities_rf_100_ph.xlsx', sheet_name='cities_100')

### Названия регионов для Datawrapper ###

In [11]:
region_name = pd.read_excel('C:/00_Projects/Lenin/Datasets/03_Other/region_name.xlsx', sheet_name='reg_name')

In [12]:
region_work_only_city = region_name[['datawrapperName', 'region']]

### Viz №1, улицы Ленина в топ-100 ###

In [13]:
top_100_streets = (work_streets
 .groupby('street', as_index=False)
 .agg({'aolevel': 'count'})
 .rename(columns={'aolevel': 'street_number'})
 .sort_values('street_number', ascending=False)
                  ).head(100) 

In [14]:
top_100_streets.head(15)

,street,street_number
117161,Центральная,25979
78562,Молодежная,17740
69629,Лесная,16501
121167,Школьная,15940
99230,Садовая,14461
83176,Новая,13812
103839,Советская,12886
80508,Набережная,11074
52898,Заречная,10701
91720,Полевая,10546


### Viz №2-3, проспекты и площади ###

In [15]:
# проспекты

top_15_avenues = (work_streets[work_streets.shortname == 'пр-кт']
                  .groupby('street', as_index=False)
                  .agg({'aolevel': 'count'})
                  .rename(columns={'street': 'avenue', 'aolevel': 'avenue_number'})
                  .sort_values('avenue_number', ascending=False)
                 )

In [16]:
top_15_avenues.head()

,avenue,avenue_number
408,Ленина,120
475,Мира,76
588,Победы,64
349,Комсомольский,37
706,Строителей,35


In [17]:
# площади

top_15_squeres = (work_streets[work_streets.shortname == 'пл']
                  .groupby('street', as_index=False)
                  .agg({'aolevel': 'count'})
                  .rename(columns={'street': 'squere', 'aolevel': 'squere_number'})
                  .sort_values('squere_number', ascending=False)
                 )

In [18]:
top_15_squeres.head(15)

,squere,squere_number
521,Ленина,216
764,Привокзальная,140
737,Победы,116
872,Советская,113
1033,Центральная,84
797,Революции,73
464,Комсомольская,57
834,Свободы,55
599,Мира,48
868,Соборная,47


### 05 Таблица: типы ленинских улиц  ###

In [88]:
(lenin_streets_strict
 .shortname.value_counts()
 .reset_index()
 .rename(columns={'index': 'street_type', 'shortname': 'number'})
)

,street_type,number
0,ул,8486
1,пер,518
2,пл,250
3,пр-кт,148
4,проезд,20
5,туп,14
6,пр-д,8
7,б-р,2
8,ш,2
9,въезд,1


### 06 Таблица: всех ленинских улиц  ###

In [20]:
lenin_streets_full.shape

(10794, 11)

In [21]:
lenin_streets_strict.shape

(9452, 11)

In [22]:
lenin_only_streets.shape

(8889, 11)

### 07 Количество ленинских улиц по регионам ###

In [104]:
len_streets_for_map = (lenin_streets_strict
                       .groupby('region', as_index=False)
                       .agg({'street': 'count'})
                       .rename(columns={'street': 'len_street_strict'})
                      )

In [105]:
len_streets_full_for_map = (lenin_streets_full
                       .groupby('region', as_index=False)
                       .agg({'street': 'count'})
                       .rename(columns={'street': 'len_street_full'})
                      )

In [106]:
len_streets_for_map = len_streets_for_map.merge(len_streets_full_for_map, on='region', how='left')

In [108]:
len_streets_for_map.head()

,region,len_street_strict,len_street_full
0,Адыгея,90,95
1,Алтай,17,19
2,Алтайский край,237,260
3,Амурская область,62,66
4,Архангельская область,39,52


## Памятники ##

In [26]:
# группируем по годам и регионам

add_year = (lenin_monuments_OKN
 .groupby(['add_year', 'region'], as_index=False)
 .agg({'monument': 'count'})
 .rename(columns={'add_year': 'year', 'monument': 'number'})
 .sort_values(['year', 'number'], ascending=[True, False])
)

In [27]:
add_year

,year,region,number
0,1957,Хабаровский край,2
3,1959,Ленинградская область,2
1,1959,Астраханская область,1
2,1959,Костромская область,1
8,1960,Москва,3
...,...,...,...
161,2015,Санкт-Петербург,1
162,2016,Крым,34
163,2016,Саха (Якутия),2
164,2017,Саха (Якутия),3


### 11 Количество памятников по регионам ###

In [125]:
# считаем памятники по регионам

all_monuments_for_map = (lenin_monuments_lst
                        .groupby('region', as_index=False)
                        .agg({'original_name': 'count'})
                        .rename(columns={'original_name': 'monument_number'})
                       )

In [126]:
# считаем памятники ОКН по регионам

OKN_monuments_for_map = (lenin_monuments_OKN
                        .groupby('region', as_index=False)
                        .agg({'monument': 'count'})
                        .rename(columns={'monument': 'OKN_number'})
                       )

In [127]:
all_monuments_for_map = all_monuments_for_map.merge(OKN_monuments_for_map, on='region', how='left')

In [129]:
all_monuments_for_map.fillna(0, inplace=True)

In [131]:
all_monuments_for_map

,region,monument_number,OKN_number
0,Адыгея,21,35.0
1,Алтай,12,14.0
2,Алтайский край,87,2.0
3,Амурская область,37,32.0
4,Архангельская область,49,0.0
...,...,...,...
80,Чечня,1,1.0
81,Чувашия,38,1.0
82,Чукотский АО,10,2.0
83,Ямало-Ненецкий АО,4,0.0


In [ ]:
#.to_excel('C:/00_Projects/Lenin/Datasets/05_for_viz/temp.xlsx', encoding='utf-8', index=False)

### Есть ли улица Ленина в каждом городе? ###

### Процент населеных пунктов, где есть ленинские улицы ###

In [50]:
cities_with_len_st = (lenin_streets_strict
                     .groupby(['region', 'mun_district', 'city_type', 'city'], as_index=False)
                     .agg({'street': 'count'})
                     .rename(columns={'street': 'len_street_number'})
                    )

In [51]:
cities_with_mon = (lenin_monuments_lst
                     .groupby(['region', 'mun_district', 'city_type', 'city'], as_index=False)
                     .agg({'original_name': 'count'})
                     .rename(columns={'original_name': 'mon_number'})
                    )

In [52]:
all_cities_with_streets = (work_streets
                           .groupby(['region', 'mun_district', 'city_type', 'city'], as_index=False)
                           .agg({'street': 'count'})
                           .rename(columns={'street': 'total_street'})
                          )

In [53]:
cities_with_len_st.shape[0] / all_cities_with_streets.shape[0]

0.09355484211452195

In [54]:
cities_with_mon.shape[0] / all_cities_with_streets.shape[0]

0.0361839211891987

In [44]:
cities_with_lenin

,region,mun_district,city_type,city,street
0,Адыгея,Адыгейск,г,Адыгейск,1
1,Адыгея,Адыгейск,х,Псекупс,1
2,Адыгея,Гиагинский,ст-ца,Гиагинская,1
3,Адыгея,Гиагинский,ст-ца,Дондуковская,1
4,Адыгея,Кошехабльский,аул,Блечепсин,1
...,...,...,...,...,...
8504,Ярославская область,Угличский,г,Углич,2
8505,Ярославская область,Ярославль,г,Ярославль,1
8506,Ярославская область,Ярославский,п,Дубки,1
8507,Ярославская область,Ярославский,п,Красные Ткачи,1


### Города с переулками Ленина, но без улиц ###

In [74]:
# агрегируем все улицы

cities_with_len_st_all = (lenin_streets_strict
                     .groupby(['region', 'mun_district', 'city_type', 'city', 'shortname_2'], as_index=False)
                     .agg({'street': 'count'})
                     .rename(columns={'street': 'len_street_number'})
                    )

In [75]:
# выбираем переулки

cities_with_len_per = cities_with_len_st_all[(
    (cities_with_len_st_all.shortname_2 == 'переулок') |
    (cities_with_len_st_all.shortname_2 == 'проезд')    
)]

In [76]:
# убираем переулки из уличного фрейма

cities_with_len_st_all = cities_with_len_st_all[~(
    (cities_with_len_st_all.shortname_2 == 'переулок') |
    (cities_with_len_st_all.shortname_2 == 'проезд')    
)]

In [77]:
cities_with_len_per = cities_with_len_per.rename(columns={'shortname_2': 'per', 'len_street_number': 'per_number'})

In [78]:
cities_with_len_per.shape

(430, 6)

In [80]:
need_cities = cities_with_len_per.merge(cities_with_len_st_all,
                             on=['region', 'mun_district', 'city_type', 'city',],
                             how='left'                            )

In [83]:
need_cities = need_cities[need_cities.len_street_number.isnull()]

In [86]:
need_cities.shape

(34, 8)

In [150]:
need_cities

,region,mun_district,city_type,city,per,per_number,shortname_2,len_street_number
18,Алтайский край,Шипуновский,с,Шипуново,переулок,1,NaN,NaN
31,Башкортостан,Калтасинский,д,Сазово,переулок,1,NaN,NaN
34,Башкортостан,Калтасинский,с,Кучаш,переулок,1,NaN,NaN
78,Волгоградская область,Кумылженский,х,Ярской 1-й,переулок,1,NaN,NaN
82,Вологодская область,Вытегорский,п,Депо,переулок,1,NaN,NaN
99,Воронежская область,Лискинский,с,Залужное,переулок,1,NaN,NaN
113,Дагестан,Дербентский,с,Куллар,переулок,1,NaN,NaN
160,Краснодарский край,Кавказский,ст-ца,Казанская,переулок,1,NaN,NaN
213,Ленинградская область,Гатчинский,гп,Тайцы,переулок,1,NaN,NaN
219,Липецкая область,Хлевенский,д,Подгорное,переулок,1,NaN,NaN


### Где любят Ленина сильнее ###

In [143]:
work = pd.read_csv('C:/00_Projects/Lenin/Datasets/04_work_data/work_region.csv')

In [147]:
weight = pd.read_csv('C:/00_Projects/Lenin/Datasets/04_work_data/work_weight.csv')

In [149]:
#.to_excel('C:/00_Projects/Lenin/Datasets/04_work_data/temp.xlsx', encoding='utf-8', index=False)